In [45]:
b = 10
num_blocks = 200
n = num_blocks*(b-1) + 1

A = spzeros(n, n)

for i = 1:num_blocks
    top_left = (i-1)*(b-1) + 1
    A[top_left:top_left+b-1, top_left:top_left+b-1] .= randn(b, b)
end

A = (A + A')/2;

B = speye(n, n);

In [47]:
@time eigs(A, nev=64, which=:LR)

  0.221355 seconds (4.60 k allocations: 2.990 MiB)


([5.92211, 5.91934, 5.7306, 5.48287, 5.4325, 5.24373, 5.23715, 5.15976, 5.15806, 5.11307  …  4.3438, 4.31362, 4.30853, 4.30031, 4.28432, 4.25878, 4.25747, 4.25498, 4.24748, 4.23991], [-3.87059e-18 8.15626e-17 … -3.34437e-15 -8.52066e-16; -2.33082e-17 -3.70661e-17 … 3.47268e-15 8.60402e-16; … ; 3.8991e-18 2.88495e-17 … -4.04913e-16 5.35424e-12; -6.86817e-18 -4.24067e-17 … 5.45989e-16 -9.81983e-12], 64, 13, 642, [0.0158025, -0.025087, -0.0190605, -0.0939225, -0.0185956, 0.0445341, -0.0301368, -0.0191006, 0.00113504, -0.0118102  …  0.163365, 0.0544653, -0.0105014, 0.077334, 0.0470517, -0.111973, -0.0908741, -0.0441456, -0.0789013, 0.0786187])

In [44]:
# grad(chol_L, B) = -1 - trace(chol_L \ B)
@time begin

function grad(chol_L::SparseArrays.CHOLMOD.Factor{Float64}, B::SparseMatrixCSC{Float64, Int64}, nu::Float64)
    tr = 0
    e_i = zeros(size(B, 2))
    e_i[1] = 1
    for i = 1:size(B, 2)
        if i > 1
            e_i[i-1] = 0
            e_i[i] = 1
        end
        tr += (chol_L \ e_i)[i]
    end
    return -1 + nu*tr
end
    
eval_f(chol_L::SparseArrays.CHOLMOD.Factor, t::Float64, nu::Float64) = -t - nu*logdet(chol_L)

curr_t = -100.
step_size = 10
nu = 1e-3

L = A - curr_t*B
init_chol_L = cholfact(L)
chol_L = copy(init_chol_L)

converged = false
    
prev_val = Inf
    
for i = 1:100
    curr_grad = grad(chol_L, B, nu)
    is_pos_def = false
        
    while !is_pos_def
        tent_t = curr_t - step_size*curr_grad
        L .= A - tent_t*B

        try
            cholfact!(chol_L, L)
            
            curr_eval = eval_f(chol_L, tent_t, nu)
            
            if curr_eval > prev_val
                step_size *= .5
                continue
            end
            
            prev_val = curr_eval
            println("curr t : $tent_t")
            is_pos_def = true
                
            if abs(curr_grad)^2 < 1e-4
                if nu < 1e-4
                    converged = true
                    break
                end
                
                nu *= .1
                prev_val = Inf
                break
            end
                
            curr_t = tent_t
            step_size *= 1.2
            break

        catch y
            if isa(y, Base.LinAlg.PosDefException)
                chol_L = copy(init_chol_L)
                step_size *= .5
            else
                error("wtf?")
            end
        end
    end
    
    if converged
        break
    end
end
    
end

curr t : -90.90057509095143
curr t : -80.08958663109283
curr t : -67.3093215656368
curr t : -52.342916356911175
curr t : -35.18004444669458
curr t : -16.69306104504443
curr t : -9.990721418128015
curr t : -9.656156349714548
curr t : -9.670938883770887
curr t : -7.237399846533563
curr t : -6.5472639506861015
curr t : -6.345936431265222
curr t : -6.286272448203201
curr t : -6.277412629059695
curr t : -6.272158267224417
curr t : -6.2714364399807145
curr t : -6.271075990936066
curr t : -6.271073798548762
curr t : -6.271074245482296
curr t : -6.270978576616144
curr t : -6.270959408656337
curr t : -6.270963127597832
curr t : -6.270962290573095
curr t : -6.270962311810957
 47.376505 seconds (3.26 M allocations: 101.795 GiB, 6.66% gc time)


In [18]:
eigs(A,nev=1,which=:SR)

([-6.17813], [-8.86227e-19; 1.70165e-18; … ; -7.09836e-17; 1.31684e-18], 1, 9, 100, [-0.00832226, -0.00582737, -0.0372743, 0.0239053, 0.0278231, 0.00543517, 0.0221169, -0.00404483, -0.0160135, -0.00235164  …  0.047227, -0.0434156, -0.00583877, 0.0929452, -0.054085, 0.0507873, -0.0536809, -0.0842467, -0.109428, 0.07352])

In [14]:
nnz(chol_L\B)

2940308

In [74]:
func(x, t, nu) = -t - nu*log(x - t)
grad(x, t, nu) = -1 + nu/(x - t)

curr_t = -100
step_size = 1

x_set = 1
nu = 1
epsilon = 1e-6
prev_val = Inf

for i=1:100
    curr_grad = grad(x_set, curr_t, nu)
    tent_t = curr_t - step_size*curr_grad
    
    if tent_t > x_set
        step_size *= .5
        continue
    end
    
    tent_val = func(x_set, tent_t, nu)
    
    if tent_val > prev_val
        step_size *= .5
        continue
    end
    
    prev_val = tent_val
    
    if abs(curr_grad)^2 < epsilon
        println("Finished optimizing iteration at nu : $nu with t $tent_val")
        if nu < 1e-4
            break
        end
        prev_val = Inf
        nu *= .1
    end
    
    curr_t = tent_t
    
    step_size *= 1.2
end

Finished optimizing iteration at nu : 1 with t 1.0484698606752908e-9
Finished optimizing iteration at nu : 0.1 with t -0.669741490289786
Finished optimizing iteration at nu : 0.010000000000000002 with t -0.9439482969657267
Finished optimizing iteration at nu : 0.0010000000000000002 with t -0.9920922447141156
Finished optimizing iteration at nu : 0.00010000000000000003 with t -0.9989789659628019
Finished optimizing iteration at nu : 1.0000000000000004e-5 with t -0.9998748707422735
